In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
#Import the required modules for test statistic calculation:
import statsmodels.stats as sm_stat
#Import the required modules for model estimation:
import statsmodels.tsa as smt
#Import the required modules for optimization:
import scipy.optimize as optimize

#We also need additional data:
import statsmodels.formula.api as smf
from  statsmodels.tsa.vector_ar.vecm import *
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
dataset=pd.read_csv('RBU_AIR.csv')

In [3]:
dataset.head(20)

,Series,Date,PM,NO,NO2,Nox,NH3,SO2,CO,Ozone,...,O Xylene,Temperature,RH,WS,WD,SR,BP,VWS,RF,Mixing Height
0,1,01-Jan-2019 - 00:00,233.39,55.37,104.26,159.63,16.20,15.99,0.24,36.77,...,3.02,16.96,55.50,0.30,232.41,134.26,1014.88,0.03,0.0,0.692308
1,2,02-Jan-2019 - 00:00,186.17,52.19,104.27,156.46,17.49,13.76,0.25,40.69,...,2.42,17.68,54.62,0.27,216.20,133.73,1014.50,0.04,0.0,0.692308
2,3,03-Jan-2019 - 00:00,299.45,172.40,155.54,327.94,10.76,27.58,0.24,35.37,...,8.83,17.32,58.46,0.20,227.11,115.84,1015.01,0.04,0.0,0.692308
3,4,04-Jan-2019 - 00:00,302.08,73.25,108.25,178.71,27.43,17.00,0.25,50.86,...,11.70,16.99,53.11,0.30,173.11,119.79,1015.65,0.04,0.0,0.692308
4,5,05-Jan-2019 - 00:00,253.62,72.53,113.37,185.90,23.85,14.53,0.24,31.54,...,3.57,17.28,56.68,0.22,218.55,135.57,1014.00,0.03,0.0,0.692308
5,6,06-Jan-2019 - 00:00,183.18,65.59,97.73,163.32,23.44,15.06,0.25,30.72,...,4.17,17.97,56.97,0.22,202.06,147.05,1012.12,0.04,0.0,0.692308
6,7,07-Jan-2019 - 00:00,179.81,109.34,128.39,234.54,22.78,18.96,0.24,27.47,...,11.41,18.54,57.03,0.22,253.37,144.08,1010.27,0.04,0.0,0.692308
7,8,08-Jan-2019 - 00:00,191.10,97.21,73.15,164.26,23.23,18.48,0.26,33.68,...,3.66,18.16,57.81,0.32,245.89,120.23,1009.84,0.06,0.0,0.692308
8,9,09-Jan-2019 - 00:00,206.16,117.88,51.19,163.19,18.68,17.79,0.24,41.03,...,3.39,18.04,58.92,0.22,240.78,110.12,1011.28,0.04,0.0,0.692308
9,10,10-Jan-2019 - 00:00,263.23,51.16,85.58,136.74,14.89,15.95,0.24,53.41,...,7.24,17.48,58.99,0.32,180.51,116.58,991.00,0.04,0.0,0.692308


In [4]:
def lag(x, n):
    if n == 0:
        return x
    if isinstance(x, pd.Series):
        return x.shift(n) 
    else:
        x = pd.Series(x)
        return x.shift(n) 

    x = x.copy()
    x[n:] = x[0:-n]
    x[:n] = np.nan
    return x

In [5]:
mod_L4_est = smf.ols(formula = 'PM ~ 1 + lag(CO, 0) + lag(CO, 1) + lag(CO, 2) + lag(CO, 3) + lag(CO, 4) + lag(CO, 5) + lag(CO, 6) + lag(CO, 7) \
                     + lag(NO, 0) + lag(NO, 1) + lag(NO, 2) + lag(NO, 3) + lag(NO, 4) + lag(NO, 5) + lag(NO, 6) + lag(NO, 7) \
                     + lag(NO2, 0) + lag(NO2, 1) + lag(NO2, 2) + lag(NO2, 3) + lag(NO2, 4) + lag(NO2, 5) + lag(NO2, 6) + lag(NO2, 7) \
                     + lag(Nox, 0) + lag(Nox, 1) + lag(Nox, 2) + lag(Nox, 3) + lag(Nox, 4) + lag(Nox, 5) + lag(Nox, 6) + lag(Nox, 7) \
                     + lag(NH3, 0) + lag(NH3, 1) + lag(NH3, 2) + lag(NH3, 3) + lag(NH3, 4) + lag(NH3, 5) + lag(NH3, 6) + lag(NH3, 7) \
                     + lag(SO2, 0) + lag(SO2, 1) + lag(SO2, 2) + lag(SO2, 3) + lag(SO2, 4) + lag(SO2, 5) + lag(SO2, 6) + lag(SO2, 7) \
                     + lag(CO, 0) + lag(CO, 1) + lag(CO, 2) + lag(CO, 3) + lag(CO, 4) + lag(CO, 5) + lag(CO, 6) + lag(CO, 7) \
                     + lag(Ozone, 0) + lag(Ozone, 1) + lag(Ozone, 2) + lag(Ozone, 3) + lag(Ozone, 4) + lag(Ozone, 5) + lag(Ozone, 6) + lag(Ozone, 7) \
                     + lag(Benzene, 0) + lag(Benzene, 1) + lag(Benzene, 2) + lag(Benzene, 3) + lag(Benzene, 4) + lag(Benzene, 5) + lag(Benzene, 6) + lag(Benzene, 7) \
                     + lag(Toluene, 0) + lag(Toluene, 1) + lag(Toluene, 2) + lag(Toluene, 3) + lag(Toluene, 4) + lag(Toluene, 5) + lag(Toluene, 6) + lag(Toluene, 7) \
                     + lag(Temperature, 0) + lag(RH, 0) + lag(WS, 0) + lag(WD, 0) + lag(SR, 0) + lag(BP, 0) + lag(VWS, 0)', data = dataset)
mod_L4_fit = mod_L4_est.fit()
print(mod_L4_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                     PM   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     74.28
Date:                Sat, 12 Feb 2022   Prob (F-statistic):          1.91e-109
Time:                        20:39:28   Log-Likelihood:                -1057.6
No. Observations:                 268   AIC:                             2275.
Df Residuals:                     188   BIC:                             2562.
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            -333.6852    

In [6]:
print(np.round(mod_L4_fit.params, 4))
print(np.round(mod_L4_fit.pvalues, 4))

Intercept     -333.6852
lag(CO, 0)      70.2264
lag(CO, 1)      -4.2518
lag(CO, 2)      -5.2627
lag(CO, 3)     -14.2393
                 ...   
lag(WS, 0)       0.6461
lag(WD, 0)       0.0512
lag(SR, 0)      -0.0633
lag(BP, 0)       0.3344
lag(VWS, 0)    109.6680
Length: 80, dtype: float64
Intercept      0.4450
lag(CO, 0)     0.0000
lag(CO, 1)     0.7933
lag(CO, 2)     0.7565
lag(CO, 3)     0.3861
                ...  
lag(WS, 0)     0.8607
lag(WD, 0)     0.0813
lag(SR, 0)     0.1453
lag(BP, 0)     0.4268
lag(VWS, 0)    0.0882
Length: 80, dtype: float64
